# Imports

In [94]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import sklearn
import tensorflow as tf
KERAS_BACKEND=tf
from tensorflow import keras
from tensorflow.keras.utils import plot_model
import gensim
# from keras_self_attention import SeqSelfAttention
# import keras
from keras.utils import np_utils
from keras.utils import to_categorical
# from keras.utils.vis_utils import model_to_dot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM
from gensim.models.keyedvectors import KeyedVectors

In [95]:
tf.__version__

'2.3.0'

# Loading In Yoga Class Data

In [96]:
 f = open("all_poses", "rb")
all_poses = pickle.load(f)
f.close()

f = open("flask_app_df", "rb")
poses_info = pickle.load(f)
f.close()

f = open("all_yoga_classes_df", "rb")
yoga_classes = pickle.load(f)
f.close()

f = open("poses_df", "rb")
class_poses = pickle.load(f)
f.close()

f = open("vinyasa_df", "rb")
more_vinyasa = pickle.load(f)
f.close()

f = open("hatha_df", "rb")
more_hatha = pickle.load(f)
f.close()

In [97]:
df = pd.concat([yoga_classes, more_vinyasa, more_hatha])
df = df.loc[df["Class Type"].isin(["Vinyasa", "Hatha", "Power", "Iyengar", "Ashtanga"])]
df.reset_index(inplace=True)

In [98]:
# average length of classes
lengths = []
for i in range(len(df["Poses"])):
    length = len(df["Poses"][i])
    lengths.append(length)
print(np.average(length))

33.0


# Tokenizing: Base Yoga Poses

In [99]:
documents = [class_list for class_list in df["Poses"]]

In [101]:
for doc in documents:
    doc.insert(0, "--")
    doc.append("--")

In [104]:
base_poses = poses_info[["Pose Name", "Base Pose"]]

In [105]:
changes = [([4, 17, 47, 50, 60], "Cycling Pose"),
    ([19, 40, 41, 44, 46, 28, 54, 67, 68, 72], "High Boat To Low Boat Flow"),
    ([64, 66, 69, 70], "Low Boat Pose"),
    ([86], "Staff Pose"), 
    ([87, 88, 117, 118], "Bound Angle Forward Bend"), 
    ([125, 135, 136, 138, 140, 141], "One Legged Bow Pose"), 
    ([216, 217, 218, 219], "Pigeon Pose"), 
    ([233, 234, 235, 236, 237, 238, 239], "Thunderbolt Pose"), 
    ([248, 269, 270, 271, 274], "Cow Pose"), 
    ([259, 260, 261, 262, 263, 364, 265, 266, 267, 268, 272, 273, 275], "Cat Pose"),
    ([291], "Hurdlers Pose"),
    ([293, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 372], "Revolved Chair Pose"), 
    ([300, 303, 306, 307, 308], "Chair Pose With Airplane Arms"),
    ([365, 369, 370], "Figure Four Pose"), 
    ([368], "Shiva Squat Pose"), 
    ([520, 521], "Supine Spinal Twist Pose"), 
    ([587, 588], "Flying Pigeon Pose"), 
    ([594, 589], "Side Crow Pose"), 
    ([599], "Baby Crow Pose"), 
    ([663, 664, 665, 666, 667, 668], "One Legged Mountain Pose"), 
    ([687, 758], "One Handed Downward Facing Dog Pose"),
    ([688, 689, 690, 759, 760], "Standing Splits Pose"), 
    ([691, 692, 693, 694, 724, 735, 736, 737, 761, 762, 763, 764], "Three Legged Downward Facing Dog Pose"), 
    ([700, 717], "Downward Facing Dog Upward Facing Dog Pose Flow"), 
    ([714], "Downward Facing Dog Pose Plank Pose Flow"), 
    ([715], "Downward Facing Dog Pose Table Top Pose Flow"), 
    ([747], "Downward Facing Dog Pose Knee To Nose"),
    ([748], "Downward Facing Dog Pose Shoulder Taps"), 
    ([749], "Downward Facing Dog Pose to Low Lunge Pose Flow"), 
    ([865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 879, 880, 881, 882, 883, 884, 885, 886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900, 901], "Easy Pose"), 
    ([924, 925, 926], "Cactus Arms"), 
    ([998], "Half Lotus Pose"), 
    ([999], "Lotus Pose"), 
    ([1016, 1017], "Eye Exercise"), ([1055, 1056, 1057, 1058, 1061], "Scorpion Pose"), 
    ([1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 1144, 1145, 1146, 1147, 1148, 1149, 1150, 1151], "Chaturanga Dandasana"),
    ([1387], "Headstand Pose Eagle Legs"), 
    ([1390], "Headstand Pose Lotus Legs"), 
    ([1392], "Headstand Pose Wide Legs"), 
    ([1395, 1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407], "Tripod Headstand Pose"), 
    ([1464, 1465, 1479, 1480, 1481, 1482], "Revolved High Lunge Pose"), 
    ([1466, 1467, 1468, 1469, 1483], "Runners Lunge Pose"), 
    ([1487, 1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498, 1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509, 1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520, 1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531, 1532, 1533], "Wide Legged Forward Fold"), 
    ([1487, 1509, 1510, 1511, 1512], "Revolved Wide Legged Forward Fold"),
    ([1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498], "Wide Legged Forward Fold With Halfway Lift"),
    ([1489, 1490, 1527], "Five Pointed Star Pose"), 
    ([1508], "Pyramid Pose"), 
    ([1534, 1535, 1536, 1537, 1538, 1539, 1540, 1541, 1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 1551, 1552, 1553, 1554, 1555, 1556, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564, 1565, 1566], "Pyramid Pose"), 
    ([1584], "Flying Lizard Pose"), 
    ([1673, 1684, 1685, 1591, 1700, 1708, 1710, 1711, 1712], "Revolved Low Lunge Pose"),
    ([1732, 1733, 1745, 1758, 1758, 1776, 1777, 1778, 1784, 1790, 1791, 1791], "One Legged Moutain Pose"), 
    ([1840, 1841, 1842, 1843, 1844, 1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852], "Easy Pose"), 
    ([1873, 1874, 1875, 1876, 1877, 1878, 1879, 1880, 1881, 1882], "Standing Side Bend Pose"), 
    ([1891, 1892, 1893, 1901, 1901, 1903, 1904, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922], "King Pigeon Pose"), 
    ([1927, 1928, 1932], "Grasshopper Pose"), 
    ([1929], "Dragonfly Pose"), 
    ([1930], "Eight Angle Pose"),
    ([1934, 1970, 1971, 1972], "One Legged Plank Pose"), ([1939], "Side Plank Pose"), 
    ([1958, 1959, 1960, 1961, 1967, 1968, 1969], "Forearm Plank Pose"), 
    ([2173], "Lotus Pose"), 
    ([2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200], "Revolved Extended Side Angle Pose"), 
    ([2189, 2195, 2196], "Revolved High Lunge Pose"), 
    ([2347, 2348, 2349, 2350, 2351, 2352, 2353, 2354, 2355, 2356, 2357, 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, 2367, 2368], "Skandasana"),
    ([2390, 2397], "Anantasana"), 
    ([2415, 2416, 2417, 2418, 2419, 2420, 2443], "Visvamitrasana Pose"), 
    ([2422, 2427, 2428, 2430, 2431, 2433, 2453, 2458, 2460], "Side Plank Pose With Leg Variation"), 
    ([2436, 2437, 2438, 2439, 2440, 2441], "Forearm Side Plank Pose"), 
    ([2465, 2466], "Wild Thing Pose"), 
    ([2482, 2483, 2484, 2486, 2487, 2488, 2489, 2480, 2492, 2493], "Half Splits Pose"), 
    ([2494], "Standing Splits Pose"), 
    ([2592, 2593, 2594], "Firefly Pose"), 
    ([2611, 2612, 2613, 2614, 2615, 2616, 2617, 2618, 2619, 2620, 2621], "Forward Fold Pose With Halfway Lift"), 
    ([2688, 2689, 2692, 2717], "Bird Of Paradise Pose"), 
    ([2700, 2701, 2703], "Revolved Hand To Big Toe Pose"), 
    ([2750, 2758], "Half Sun Salutation"), 
    ([2759], "Second Half Of Sun Salutation"), 
    ([2771, 2772], "Banana Pose"), 
    ([2795], "Supine Spinal Twist Pose"), 
    ([2797, 2798, 2833, 2848, 2849, 2850], "Tiger Pose"), 
    ([2799, 2816, 2827], "Table Top Knee To Nose Flow"), 
    ([2800, 2817], "Child Pose Table Top Pose Flow"), 
    ([2801, 2831, 2834, 2837, 2840, 2841, 2842], "Table Top Pose With One Leg Extended Back"), 
    ([2809, 2810, 2811, 2812, 2813, 2814, 2815], "Revolved Table Top Pose"), 
    ([2852, 2853, 2854, 2855], "Table Top Balancing Pose, Opposite Arm and Leg Extended"), 
    ([2968, 2972, 2983, 2974, 2975, 2976, 2977, 3005, 3006, 3007, 3008], "Revolved Triangle Pose"), 
    ([3023, 3024, 3025, 3026, 3031, 3032, 3038, 3039, 3040], "Reverse Table Top Pose"), 
    ([3037], "Flip The Dog Pose"), 
    ([3046, 3047, 3048, 3049, 3050, 3051, 3052, 3053, 3054, 3055, 3056, 3057, 3058, 3059, 3060, 3061, 3062, 3063], "Anantasana"), 
    ([3064, 3065, 3066, 3067, 3070, 3071, 3072, 3073, 3074, 3075, 3079], "Warrior Pose I"), 
    ([3068, 3069, 3076, 3077, 3078], "Humble Warrior Pose"), 
    ([3106, 3107, 3108, 3109, 3110, 3111, 3112, 3113, 3114, 3115, 3116, 3117, 3118, 3119, 3120, 3121], "Warrior Pose II"), 
    ([3122, 3123, 3124, 3125, 3126, 3127, 3128, 3129, 3130, 3131, 3132, 3133, 3134, 3135, 3136, 3137, 3138, 3139, 3140, 3141, 3142, 3143, 3144], "Warrior Pose III"), 
    ([3134], "Shiva Squat Pose"), 
    ([3131, 3132, 3133], "Airplane Pose"), 
    ([3181, 3182, 3183, 3184, 3185, 3186, 3187, 3188, 3189, 3190], "Half Wind Release Pose")]

In [106]:
for i in changes:
    base_poses.loc[i[0], "Base Pose"] = i[1]

In [107]:
len(base_poses["Base Pose"].unique())

170

In [108]:
base_documents = []
for i in range(len(documents)):
    temp_df = pd.DataFrame(documents[i], columns=["0"])
    temp_df = pd.merge(temp_df, base_poses, how="left", left_on="0", right_on="Pose Name")
    temp_df = temp_df.dropna(how="any")
    new_doc = [pose for pose in temp_df["Base Pose"]]
    base_documents.append(new_doc)

# Word2Vec Embeddings

In [109]:
embeddings = gensim.models.Word2Vec(base_documents, size=100, window=5, min_count=1, sg=1)

In [110]:
embeddings_size = len(list(embeddings.wv.vocab.items()))

In [111]:
embeddings_size

169

In [112]:
print(embeddings.similarity("Corpse Pose", "Extended Side Angle Pose"))
print(embeddings.similarity("Corpse Pose", "Pigeon Pose"))
print(embeddings.similarity("Corpse Pose", "Child Pose"))
print(embeddings.similarity("Corpse Pose", "Wind Release Pose"))

0.112880096
0.24852465
0.38582063
0.8037262


In [114]:
embedding_matrix = np.zeros((len(embeddings.wv.vocab)+1, 100))
for i in range(len(embeddings.wv.vocab)):
    embedding_vector = embeddings.wv[embeddings.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [115]:
embedding_dim = embeddings.vector_size

# LSTM Network


In [117]:
tokens = [pose for yogaclass in base_documents for pose in yogaclass]
reversed_tokens = tokens[::-1]
print("Total tokens: ", len(tokens))
print("Unique tokens: ", len(set(tokens)))

Total tokens:  1624681
Unique tokens:  169


In [118]:
length = 3 + 1
sequences = list()
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    sequences.append(seq)
print("Total Sequences:", len(sequences))

Total Sequences: 1624677


In [129]:
# reversed_sequences = list()
# for i in range(length, len(reversed_tokens)):
#     rev_seq = reversed_tokens[i-length:i]
#     reversed_sequences.append(seq)
# print("Total Reversed Sequences:", len(reversed_sequences))

Total Reversed Sequences: 1624677


In [119]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
token_sequences = tokenizer.texts_to_sequences(sequences)
vocab_size = len(tokenizer.word_index) + 1

In [130]:
# tokenizer2 = Tokenizer()
# tokenizer2.fit_on_texts(reversed_sequences)
# reversed_token_sequences = tokenizer2.texts_to_sequences(reversed_sequences)

In [48]:
vocab_size

14675

In [120]:
X = np.asarray([np.asarray(x[:-1]) for x in token_sequences])
y = [y[-1] for y in token_sequences]
y = to_categorical(y, num_classes=vocab_size)
seq_length=len(X[1])

In [131]:
# X2 = np.asarray([np.asarray(x[:-1]) for x in reversed_token_sequences])
# y2 = [y[-1] for y in reversed_token_sequences]
# y2 = to_categorical(y2, num_classes=vocab_size)

In [175]:
def create_model():
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], trainable=False))
    model.add(Bidirectional(LSTM(200, return_sequences=True)))
    model.add(Dropout(0.05))
    model.add(Bidirectional(LSTM(200, return_sequences=True)))
    model.add(Dropout(0.05))
    model.add(Bidirectional(LSTM(200)))
    model.add(Dropout(0.05))
    model.add(Dense(100, activation="relu"))
    model.add(Dense(vocab_size, activation="softmax"))
    print(model.summary())
    return model

In [183]:
def fit_model(model, X, y):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X, y, batch_size=128, epochs=100, verbose=10)

In [184]:
forward_model = create_model()
backward_model = create_model()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, None, 100)         17000     
_________________________________________________________________
bidirectional_67 (Bidirectio (None, None, 400)         481600    
_________________________________________________________________
dropout_54 (Dropout)         (None, None, 400)         0         
_________________________________________________________________
bidirectional_68 (Bidirectio (None, None, 400)         961600    
_________________________________________________________________
dropout_55 (Dropout)         (None, None, 400)         0         
_________________________________________________________________
bidirectional_69 (Bidirectio (None, 400)               961600    
_________________________________________________________________
dropout_56 (Dropout)         (None, 400)             

In [185]:
fit_model(forward_model, X, y)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [ ]:
# fit_model(backward_model, X2, y2)

In [186]:
model.save('forward_model.h5')
# model.save('backward_model.h5')

In [ ]:
backward_model = load_model('backward_model.h5')

In [187]:
def generate_class(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    for _ in range(n_words): # change this to arbitrarily large? so it keeps running until "--"? While true? 
        encoded = tokenizer.texts_to_sequences([in_text])
        encoded = [pad_sequences([encoded[0]], maxlen=seq_length, truncating='pre')]
        yhat = np.argmax(model.predict(encoded), axis=-1)
        out_word = ""
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        if out_word == "--":
            break
        in_text.append(out_word)
        result.append(out_word)
    return result

In [ ]:
possible_seeds = []

In [191]:
print(base_poses["Base Pose"].unique())

['Boat Pose' 'Cycling Pose' 'High Boat To Low Boat Flow' 'Low Boat Pose'
 'Bound Angle Pose' 'Staff Pose' 'Bound Angle Forward Bend' 'Bow Pose'
 'One Legged Bow Pose' 'Bridge Pose' 'Camel Pose' 'Pigeon Pose'
 'Thunderbolt Pose' 'Cow Pose' 'Cat Cow Pose' 'Cat Pose'
 'Cat Tail Pulling Pose' 'Center Splits Pose' 'Hurdlers Pose'
 'Revolved Chair Pose' 'Chair Pose' 'Chair Pose With Airplane Arms'
 'Figure Four Pose' 'Shiva Squat Pose' 'Child Pose' 'Cobra Pose'
 'Corpse Pose' 'Supine Spinal Twist Pose' 'Cow Face Pose' 'Crane Pose'
 'Flying Pigeon Pose' 'Side Crow Pose' 'Baby Crow Pose' 'Crow Pose'
 'Dancer Pose' 'One Legged Mountain Pose' 'Dolphin Pose'
 'Downward Facing Dog Pose' 'One Handed Downward Facing Dog Pose'
 'Standing Splits Pose' 'Three Legged Downward Facing Dog Pose'
 'Downward Facing Dog Upward Facing Dog Pose Flow'
 'Downward Facing Dog Pose Plank Pose Flow'
 'Downward Facing Dog Pose Table Top Pose Flow'
 'Downward Facing Dog Pose Knee To Nose'
 'Downward Facing Dog Pose Sho

In [193]:
second_half = generate_class(forward_model, tokenizer, seq_length, np.random.choice(["Half Moon Pose"]), 50)

AttributeError: 'numpy.str_' object has no attribute 'append'

In [ ]:
first_half = generate_class(backward_model, tokenizer2, seq_length, np.random.choice(possible_seeds), 50)

In [ ]:
first_half = first_half.reversed()

In [ ]:
class = first_half + second_half